In [1]:
import os
from getpass import getpass

In [2]:
HUGGINGFACEHUB_API_TOKEN = getpass()

 ·····································


In [20]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN

In [3]:
from langchain import HuggingFaceHub, PromptTemplate, LLMChain

In [78]:
template='''Answer the question using the provided context concisely and accurately in one sentence. Strive for an exact match to expected responses, focusing on clarity and relevance to achieve high F1, exact match, and recall scores in grading.
Question: {question}
Context: {context}
Answer:
'''

In [79]:
template

'Answer the question using the provided context concisely and accurately in one sentence. Strive for an exact match to expected responses, focusing on clarity and relevance to achieve high F1, exact match, and recall scores in grading.\nQuestion: {question}\nContext: {context}\nAnswer:\n'

In [80]:
prompt = PromptTemplate(template=template, input_variables=['question', 'context'])

In [81]:
prompt

PromptTemplate(input_variables=['context', 'question'], template='Answer the question using the provided context concisely and accurately in one sentence. Strive for an exact match to expected responses, focusing on clarity and relevance to achieve high F1, exact match, and recall scores in grading.\nQuestion: {question}\nContext: {context}\nAnswer:\n')

In [82]:
llm = HuggingFaceHub(repo_id='mistralai/Mixtral-8x7B-Instruct-v0.1',
                    model_kwargs={'temperature': 0.6, 'max_length': 25})

In [83]:
llm

HuggingFaceHub(client=<InferenceClient(model='mistralai/Mixtral-8x7B-Instruct-v0.1', timeout=None)>, repo_id='mistralai/Mixtral-8x7B-Instruct-v0.1', task='text-generation', model_kwargs={'temperature': 0.6, 'max_length': 25})

In [23]:
import torch
from sentence_transformers import SentenceTransformer
from torch.nn import DataParallel
import numpy as np

class MistralEmbedder:
    def __init__(self, model_name="Salesforce/SFR-Embedding-Mistral", device='cuda'):
#         device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = SentenceTransformer(model_name, device=device)
    
    def embed_documents(self, texts):
        """
        Generate embeddings for a list of text documents using the Mistral model.
        
        Parameters:
        - texts (List[str]): A list of texts to embed.
        
        Returns:
        - List of embeddings as numpy arrays.
        """
        # Ensure texts is a list for batch processing
        if isinstance(texts, str):
            texts = [texts]
        
        # Generate embeddings and return them as numpy arrays
        embeddings = self.model.encode(texts)
        return embeddings
    def embed_query(self, query):
        """
        Generate an embedding for a single query string using the Mistral model.
        
        Parameters:
        - query (str): The query string to embed.
        
        Returns:
        - A numpy array representing the embedding of the query.
        """
        # Generate the embedding for the query
        embedding = self.model.encode(query, convert_to_numpy=True)

        # Convert numpy array to list if necessary
        embedding_list = embedding.tolist() if isinstance(embedding, np.ndarray) else embedding

        # Return the embedding
        return embedding_list

# Initialize the embedding wrapper
mistral_embedder = MistralEmbedder()

Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.89it/s]


In [30]:
from langchain_community.vectorstores import Chroma

# vectorstore = Chroma(persist_directory='sfr-mistralDB', embedding_function=mistral_embedder)

In [49]:
retriever = vectorstore.as_retriever()
query = "What is LTI"
docs = retriever.get_relevant_documents(query)

In [59]:
question = "What is Buggy?"

In [63]:
another_document = retriever.get_relevant_documents(query)

In [64]:
print(another_document)

[Document(page_content='by actual classroom courses, not credit given for research or independent study.\n\nAn “LTI course ” is any 12 -unit course with a number of 11 -XXX; a  6-unit course with 11 -XXX\n\ncounts as one- half of an LTI course. Unless otherwise specified, "course" means an actual\n\nclassroom course, not  credit given for research or independent study. Note that the LTI  allow s\n\nany one MLD (10 -XXX) graduate course to count as an "LTI course".'), Document(page_content='MLT Graduate Student Handbook  Page 17\n\n4.4 Definitions of LTI Terminology\n\nWe define here some of the terms as used in this handbook:\n\nAn “LTI course” is any 12 -unit course with a number of 11 -XXX; a 6 -unit course with an\n\n11-XXX counts as 1/2 of an LTI course. Unless otherwise specified, "course" means an\n\nactual classroom course, not  credit given for research or independent study. Note that\n\nwe will allow any one MLD (10 -xxx) graduate- level course to count as an “LTI course.”'), 

In [84]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnablePick
from langchain_core.prompts.chat import HumanMessagePromptTemplate, PromptTemplate
# Chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# print(llm.invoke(question))

In [85]:
import os
from tqdm import tqdm
import re


# Load questions from questions.txt
with open("SubmissionData/test/questions.txt", "r") as f:
    questions = [line.strip() for line in f.readlines()]

# Output directory for answers
output_dir = "SubmissionData/system_outputs/"
answer_file = os.path.join(output_dir, "MistralInstruct8x7.txt")

# A function to extract the answer from the model's response
def extract_answer(response):
    match = re.search(r'Answer:\s*(.*?)\s*$', response, re.DOTALL)
    return match.group(1) if match else "No answer found."

# Run the question-answering loop and save answers
answers = []
with tqdm(total=len(questions), desc="Answering questions") as progress_bar:
    with open(answer_file, "w") as f:
        for question in questions:
            response = qa_chain.invoke(question)
            # Extract the concise answer from the response
            concise_answer = extract_answer(response)
            f.write(concise_answer + "\n")
            f.write(concise_answer.replace("\n",""))
            answers.append(concise_answer)
            progress_bar.update(1)

Answering questions:  77%|███████▋  | 61/79 [00:11<00:03,  5.41it/s]


HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/mistralai/Mixtral-8x7B-Instruct-v0.1 (Request ID: lSz5DdCDiuQfon-Y-qQea)

Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate

In [41]:
import pickle

# Replace 'your_file_path.pkl' with the path to your .pkl file
file_path = 'splitDocuments.pkl'

with open(file_path, 'rb') as file:
    contents = pickle.load(file)

# Extracting page_content from each document object
docs_texts = []
for sublist in contents:
    for doc in sublist:  # Assuming each element in sublist is a document object
        docs_texts.append(doc.page_content)

In [42]:
from langchain_community.vectorstores import Chroma

# Initialize all_texts with leaf_texts
all_texts = docs_texts.copy()

# Build the vector store with Chroma
vectorstore = Chroma.from_texts(texts=all_texts, embedding=mistral_embedder, persist_directory="MistralEmbed/")

In [67]:
# retriever = vectorstore.as_retriever()
query = "How does the MSAII program relate to its predecessor?"
docs = retriever.get_relevant_documents(query)
print(docs)

[Document(page_content='2\n\n2. Related Works'), Document(page_content='2. Related Works'), Document(page_content='2 Related Works'), Document(page_content='(2)')]


In [68]:
vectorstore.get()

{'ids': ['0002cbac-ca3b-4a3f-9982-58a80075cb49',
  '0004990d-4394-48fe-bf80-6524adb1f6d7',
  '0007653b-caec-41c8-ad22-f6e8900eab97',
  '0007d2c6-6036-4d55-8824-12eda32bd628',
  '0008c890-7405-47a3-960b-95425afdf6d5',
  '000e2d2b-19e7-4e79-a370-f60957a98112',
  '0010210a-d26d-4289-a38e-434e2a17ce38',
  '00107455-8fb5-45a1-9708-2278381e443c',
  '001544c7-6a76-4916-bb65-e5e63eb8e807',
  '0016a46c-4188-4049-811c-d783c0fd38df',
  '0017a7a6-8b8f-49e4-8b77-128dd413e95f',
  '0018b169-c7df-4dc8-a8e4-053e80f14e07',
  '001c47eb-4fce-4276-9231-9acea3b2b021',
  '002432be-63ab-4dd4-b9f2-91060de01f33',
  '002449c2-6fa8-4707-85eb-e29a43a99499',
  '002602f4-a56e-41b5-94f3-0b05c0c19d63',
  '00264dfc-6d1f-4fa6-bc98-d4a1a6d0417b',
  '0026cc72-fe47-410d-89f4-ad7ad1968c94',
  '002702f1-cd22-45df-ba9e-32cefc637e70',
  '0027a12d-f173-4060-9820-74c60ed5bb70',
  '0028e487-5ddd-4b1e-82e9-dcf472056137',
  '002a1b1a-27a1-444c-9441-a11ea339f366',
  '002c2373-99d8-4b69-92ba-de5ba3287053',
  '002fcb7a-6a3c-46e7-ac58-